<a href="https://colab.research.google.com/github/Gouthamyeluri/malaria-detection/blob/main/malriya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gouthamyeluri","key":"ea5ada6d5fa7264ed6679a4c0d542dd6"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

 97% 658M/675M [00:06<00:00, 96.1MB/s]
100% 675M/675M [00:06<00:00, 107MB/s] 


In [7]:
!mkdir malaria_data

In [8]:
!unzip "/content/cell-images-for-detecting-malaria.zip" -d "/content/malaria_data"

Streaming output truncated to the last 5000 lines.
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_116.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_118.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_126.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_134.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_141.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_168.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_175.png  
  inflating: /content/malaria_data/cell_images/cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_183.png  
  inflating: /content/malaria

In [10]:
!mkdir final_data

In [1]:
import os
find_len=os.listdir('/content/malaria_data/cell_images/Parasitized')


In [2]:
len(find_len)

13780

In [8]:
import os
import shutil
for i in range(0,2500):
  shutil.move("/content/malaria_data/cell_images/cell_images/Parasitized/{}".format(find_len[i]) , "/content/final_data/train/Parasitized")

for i in range(2500,3200):
  shutil.move("/content/malaria_data/cell_images/cell_images/Parasitized/{}".format(find_len[i]) , "/content/final_data/test/Parasitized")



In [10]:
find_len1=os.listdir('/content/malaria_data/cell_images/Uninfected')

In [11]:
len(find_len1)

13780

In [12]:
import os
import shutil
for i in range(0,2500):
  shutil.move("/content/malaria_data/cell_images/cell_images/Uninfected/{}".format(find_len1[i]) , "/content/final_data/train/Uninfected")
for i in range(2500,3200):
  shutil.move("/content/malaria_data/cell_images/cell_images/Uninfected/{}".format(find_len1[i]) , "/content/final_data/test/Uninfected")



In [13]:
find_len11=os.listdir('/content/final_data/test/Uninfected')

In [15]:
len(find_len11)

700

In [16]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [17]:
IMAGE_SIZE = [224, 224]

train_path = '/content/final_data/train'
valid_path = '/content/final_data/test'

In [18]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 1s 0us/step


In [20]:
for layer in vgg.layers:
    layer.trainable = False

In [22]:
x = Flatten()(vgg.output)

In [56]:
folders = glob('/content/final_data/train/*')
folders

['/content/final_data/train/Uninfected',
 '/content/final_data/train/Parasitized']

In [57]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

In [58]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [59]:
from tensorflow.keras.layers import MaxPooling2D


In [60]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(2,activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 50176)            

In [61]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [62]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
import os
import shutil

os.listdir("/content/final_data/train") 
shutil.rmtree("/content/final_data/train/.ipynb_checkpoints") 

In [72]:
training_set = train_datagen.flow_from_directory('/content/final_data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 5000 images belonging to 2 classes.


In [73]:
import os
import shutil

os.listdir("/content/final_data/test") 
shutil.rmtree("/content/final_data/test/.ipynb_checkpoints") 

In [76]:
test_set = test_datagen.flow_from_directory('/content/final_data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1400 images belonging to 2 classes.


In [77]:
len(test_set)


44

In [78]:
model.fit_generator(training_set,validation_data=test_set,epochs=15,steps_per_epoch=len(training_set),validation_steps=len(test_set))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
157/157 [==============================] - 225s 1s/step - loss: 1.0829 - accuracy: 0.5236 - val_loss: 0.6613 - val_accuracy: 0.6164
Epoch 2/15
157/157 [==============================] - 222s 1s/step - loss: 0.6590 - accuracy: 0.6008 - val_loss: 0.6561 - val_accuracy: 0.5793
Epoch 3/15
157/157 [==============================] - 222s 1s/step - loss: 0.5926 - accuracy: 0.7078 - val_loss: 0.3673 - val_accuracy: 0.8579
Epoch 4/15
157/157 [==============================] - 222s 1s/step - loss: 0.3349 - accuracy: 0.8716 - val_loss: 0.2216 - val_accuracy: 0.9143
Epoch 5/15
157/157 [==============================] - 222s 1s/step - loss: 0.2563 - accuracy: 0.9126 - val_loss: 0.2215 - val_accuracy: 0.9200
Epoch 6/15
157/157 [==============================] - 222s 1s/step - loss: 0.2089 - accuracy: 0.9268 - val_loss: 0.1929 - val_accuracy: 0.9286
Epoch 7/15
157/157 [==============================] - 222s 1s/step - loss: 0.1849 - accuracy: 0.9331 - val_loss: 0.2095 - val_accuracy: 0.9314

In [79]:

from fastai.vision import *
import torch.nn as nn
import torch.nn.functional as F

In [84]:
from tensorflow.keras.models import load_model

model.save('model_vgg19.h5')

In [86]:
y_pred = model.predict(test_set)

In [87]:
y_pred

array([[9.927887e-01, 7.211239e-03],
       [9.984475e-01, 1.552496e-03],
       [4.316327e-03, 9.956837e-01],
       [9.948008e-01, 5.199190e-03],
       ...,
       [9.998316e-01, 1.684210e-04],
       [2.942839e-02, 9.705716e-01],
       [9.985551e-01, 1.444925e-03],
       [5.780899e-02, 9.421910e-01]], dtype=float32)

In [88]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [89]:
model=load_model('model_vgg19.h5')

In [99]:
img=image.load_img('/content/malaria_data/cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png',target_size=(224,224))

In [100]:
x=image.img_to_array(img)
x


array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0

In [101]:

x.shape

(224, 224, 3)

In [102]:
x=x/255


In [103]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [104]:
model.predict(img_data)

array([[1., 0.]], dtype=float32)

In [105]:
a=np.argmax(model.predict(img_data), axis=1)

In [107]:
if(a==1):
    print("Uninfected")
else:
    print("Infected")


Infected
